In [13]:
import pickle

from mlscorecheck.utils import ProblemSolver, generate_problem, generate_problem_tp0
from mlscorecheck.core import score_function_set, sqrt
from mlscorecheck.core import Interval, union, reduce_solutions, collect_with_respect_to_vars

import sympy

In [14]:
scores = score_function_set()

In [15]:
score0 = 'acc'
score1 = 'npv'

score0_fun = scores[score0]
score1_fun = scores[score1]

In [16]:
tp = sympy.Symbol('tp')
tn = sympy.Symbol('tn')
fp = sympy.Symbol('fp')
fn = sympy.Symbol('fn')
p = sympy.Symbol('p')
n = sympy.Symbol('n')

symbols = {'tp': tp,
            'tn': tn,
            'fp': fp,
            'fn': fn,
            'p': p,
            'n': n}

score0_sym = sympy.Symbol(score0)
score1_sym = sympy.Symbol(score1)

In [17]:
score0_args = score0_fun.__code__.co_varnames[:score0_fun.__code__.co_kwonlyargcount]
score1_args = score1_fun.__code__.co_varnames[:score1_fun.__code__.co_kwonlyargcount]

In [18]:
score0_args, score1_args

(('tp', 'tn', 'p', 'n'), ('tn', 'fn'))

In [22]:
score0_exp = score0_fun(**{key: val for key, val in symbols.items() if key in score0_args})
score1_exp = score1_fun(**{key: val for key, val in symbols.items() if key in score1_args})
score0_eq = score0_sym - score0_exp
score1_eq = score1_sym - score1_exp
n_eq = n - tn - fp
p_eq = p - tp - fn

In [30]:
score1_eq2 = score1_sym*(symbols['tn'] + symbols['fn']) - symbols['tn']

In [23]:
score0_eq, score1_eq

(acc - (tn + tp)/(n + p), npv - tn/(fn + tn))

In [24]:
score0_exp, score1_exp

((tn + tp)/(n + p), tn/(fn + tn))

In [25]:
def transform_to_polynom(expression):
    _, denom = expression.as_numer_denom()
    return sympy.simplify(expression * denom)

In [26]:
def special_cases(equations, variables, order='grlex'):
    
    peq = [transform_to_polynom(eq) for eq in equations]
    gb = sympy.GroebnerBasis(peq, *variables, order=order)
    if not gb.is_zero_dimensional:
        return None
    
    edge_cases = {}
    
    i = 0
    
    for basis in gb:
        terms = collect_with_respect_to_vars(basis, variables)
        for term, coeff in terms.items():
            print(coeff)
            results0 = sympy.solve(coeff, score0_sym, dict=True)
            results1 = sympy.solve(coeff, score1_sym, dict=True)
            
            if len(str(results0)) < len(str(results1)):
                results = results0
            else:
                results = results1
            
            print(results)
            for res in results:
                key = str(res)
                if key in edge_cases:
                    continue
                edge_cases[key] = res
            i = i + 1
    print(edge_cases)
    
    #return edge_cases
    
    for key, val in edge_cases.items():
        subs = val
        ec_equations = [eq.subs(subs) for eq in equations]
        print(val)
        print(ec_equations)
        print(sympy.solve(ec_equations, tp, tn, fp, fn, check=False, dict=True))
        
        """
        ec_gb = sympy.GroebnerBasis(ec_equations, *variables, order=order)
        print(ec_gb)
        zd = ec_gb.is_zero_dimensional
        print(zd)
        if zd:
            print(sympy.solve(ec_equations, *variables, dict=True))
        """
    
        

In [27]:
score1_eq

npv - tn/(fn + tn)

In [31]:
sympy.GroebnerBasis([score0_eq, score1_eq2, n_eq, p_eq], [tp, tn, fp, fn])

GroebnerBasis([tp + (-acc*n*npv + acc*n - acc*npv*p + acc*p - npv*p)/(2*npv - 1), tn + (-acc*n*npv - acc*npv*p + npv*p)/(2*npv - 1), fp + (acc*n*npv + acc*npv*p - 2*n*npv + n - npv*p)/(2*npv - 1), fn + (acc*n*npv - acc*n + acc*npv*p - acc*p - npv*p + p)/(2*npv - 1)], tp, tn, fp, fn, domain='ZZ(p,n,acc,npv)', order='lex')

In [28]:
edge_cases = special_cases([score0_eq, score1_eq, n_eq, p_eq], [tp, tn, fp, fn])

2*npv - 1
[]
-acc*n*npv + acc*n - acc*npv*p + acc*p - npv*p
[{acc: -npv*p/(n*npv - n + npv*p - p)}]
2*npv - 1
[]
-acc*n*npv - acc*npv*p + npv*p
[{npv: 0}]
2*npv - 1
[]
acc*n*npv + acc*npv*p - 2*n*npv + n - npv*p
[{npv: -n/(acc*n + acc*p - 2*n - p)}]
2*npv - 1
[]
acc*n*npv - acc*n + acc*npv*p - acc*p - npv*p + p
[{npv: 1}]
{'{acc: -npv*p/(n*npv - n + npv*p - p)}': {acc: -npv*p/(n*npv - n + npv*p - p)}, '{npv: 0}': {npv: 0}, '{npv: -n/(acc*n + acc*p - 2*n - p)}': {npv: -n/(acc*n + acc*p - 2*n - p)}, '{npv: 1}': {npv: 1}}
{acc: -npv*p/(n*npv - n + npv*p - p)}
[-npv*p/(n*npv - n + npv*p - p) - (tn + tp)/(n + p), npv - tn/(fn + tn), -fp + n - tn, -fn + p - tp]
[{fn: p, fp: (n*npv - n + npv*p)/(npv - 1), tn: -npv*p/(npv - 1), tp: 0}]
{npv: 0}
[acc - (tn + tp)/(n + p), -tn/(fn + tn), -fp + n - tn, -fn + p - tp]
[{fn: -acc*n - acc*p + p, fp: n, tn: 0, tp: acc*n + acc*p}]
{npv: -n/(acc*n + acc*p - 2*n - p)}
[acc - (tn + tp)/(n + p), -n/(acc*n + acc*p - 2*n - p) - tn/(fn + tn), -fp + n - tn, -fn

In [151]:
for key, val in edge_cases.items():
    print(key, val)

{dor: 0} {dor: 0}
{n: -p/mcc**2} {n: -p/mcc**2}
{p: 0} {p: 0}
{dor: 1} {dor: 1}
{n: -mcc**2*p} {n: -mcc**2*p}
{dor: mcc*(mcc*n**2 - mcc*p**2 - sqrt(mcc**2*n**4 + 2*mcc**2*n**2*p**2 + 4*mcc**2*n*p**3 + mcc**2*p**4 - 4*n**2*p**2 - 4*n*p**3))/(2*n*p*(mcc**2 - 1))} {dor: mcc*(mcc*n**2 - mcc*p**2 - sqrt(mcc**2*n**4 + 2*mcc**2*n**2*p**2 + 4*mcc**2*n*p**3 + mcc**2*p**4 - 4*n**2*p**2 - 4*n*p**3))/(2*n*p*(mcc**2 - 1))}
{dor: mcc*(mcc*n**2 - mcc*p**2 + sqrt(mcc**2*n**4 + 2*mcc**2*n**2*p**2 + 4*mcc**2*n*p**3 + mcc**2*p**4 - 4*n**2*p**2 - 4*n*p**3))/(2*n*p*(mcc**2 - 1))} {dor: mcc*(mcc*n**2 - mcc*p**2 + sqrt(mcc**2*n**4 + 2*mcc**2*n**2*p**2 + 4*mcc**2*n*p**3 + mcc**2*p**4 - 4*n**2*p**2 - 4*n*p**3))/(2*n*p*(mcc**2 - 1))}
{n: mcc**2*p*(dor - 2)/(dor*mcc**2 - 2*dor + mcc**2 + 1)} {n: mcc**2*p*(dor - 2)/(dor*mcc**2 - 2*dor + mcc**2 + 1)}
{mcc: -sqrt(-p/n)} {mcc: -sqrt(-p/n)}
{mcc: sqrt(-p/n)} {mcc: sqrt(-p/n)}
{n: 0} {n: 0}
{dor: mcc**2*(n + p)/(mcc**2*n - mcc**2*p + 2*p)} {dor: mcc**2*(n + p)/(mcc**2

In [114]:
dor = score0_sym
mcc = score1_sym

In [119]:
edge_cases[list(edge_cases.keys())[5]]

{dor: mcc*(mcc*n**2 - mcc*p**2 - sqrt(mcc**2*n**4 + 2*mcc**2*n**2*p**2 + 4*mcc**2*n*p**3 + mcc**2*p**4 - 4*n**2*p**2 - 4*n*p**3))/(2*n*p*(mcc**2 - 1))}

In [120]:
tmp = sympy.simplify((edge_cases[list(edge_cases.keys())[5]][score0_sym] - score0_sym) * (2*n*p*(mcc**2 - 1)))

In [124]:
print(sympy.expand(tmp))

-2*dor*mcc**2*n*p + 2*dor*n*p + mcc**2*n**2 - mcc**2*p**2 - mcc*sqrt(mcc**2*n**4 + 2*mcc**2*n**2*p**2 + 4*mcc**2*n*p**3 + mcc**2*p**4 - 4*n**2*p**2 - 4*n*p**3)


In [126]:
tmp2 = ((-2*dor*mcc**2*n*p + 2*dor*n*p + mcc**2*n**2 - mcc**2*p**2) / mcc)**2 - mcc**2*n**4 + 2*mcc**2*n**2*p**2 + 4*mcc**2*n*p**3 + mcc**2*p**4 - 4*n**2*p**2 - 4*n*p**3

In [128]:
tmp3 = tmp2.subs({dor: (tn*tp/(n - tn)/(p - tp))})

In [134]:
tmp3

-mcc**2*n**4 + 2*mcc**2*n**2*p**2 + 4*mcc**2*n*p**3 + mcc**2*p**4 - 4*n**2*p**2 - 4*n*p**3 + (mcc**2*n**2 - 2*mcc**2*n*p*tn*tp/((n - tn)*(p - tp)) - mcc**2*p**2 + 2*n*p*tn*tp/((n - tn)*(p - tp)))**2/mcc**2

In [140]:
num, denom = sympy.simplify(sympy.expand(tmp3)).as_numer_denom()

In [142]:
tmp4 = sympy.simplify(sympy.simplify(sympy.expand(tmp3)) * denom)

In [145]:
tmp5 = sympy.expand(tmp4)

In [149]:
tmp5

-4*mcc**4*n**5*p**3*tn*tp + 8*mcc**4*n**5*p**2*tn*tp**2 - 4*mcc**4*n**5*p*tn*tp**3 + 4*mcc**4*n**4*p**6 - 12*mcc**4*n**4*p**5*tp + 12*mcc**4*n**4*p**4*tp**2 + 8*mcc**4*n**4*p**3*tn**2*tp - 4*mcc**4*n**4*p**3*tp**3 - 16*mcc**4*n**4*p**2*tn**2*tp**2 + 8*mcc**4*n**4*p*tn**2*tp**3 + 2*mcc**4*n**3*p**7 - 12*mcc**4*n**3*p**6*tn - 6*mcc**4*n**3*p**6*tp + 40*mcc**4*n**3*p**5*tn*tp + 6*mcc**4*n**3*p**5*tp**2 - 44*mcc**4*n**3*p**4*tn*tp**2 - 2*mcc**4*n**3*p**4*tp**3 - 4*mcc**4*n**3*p**3*tn**3*tp + 4*mcc**4*n**3*p**3*tn**2*tp**2 + 16*mcc**4*n**3*p**3*tn*tp**3 + 8*mcc**4*n**3*p**2*tn**3*tp**2 - 4*mcc**4*n**3*p**2*tn**2*tp**3 - 4*mcc**4*n**3*p*tn**3*tp**3 - 6*mcc**4*n**2*p**7*tn + 12*mcc**4*n**2*p**6*tn**2 + 18*mcc**4*n**2*p**6*tn*tp - 44*mcc**4*n**2*p**5*tn**2*tp - 18*mcc**4*n**2*p**5*tn*tp**2 + 52*mcc**4*n**2*p**4*tn**2*tp**2 + 6*mcc**4*n**2*p**4*tn*tp**3 - 4*mcc**4*n**2*p**3*tn**3*tp**2 - 20*mcc**4*n**2*p**3*tn**2*tp**3 + 4*mcc**4*n**2*p**2*tn**3*tp**3 + 6*mcc**4*n*p**7*tn**2 - 4*mcc**4*n*p**6*t

In [152]:
sympy.solve([tmp5, score1_eq, p_eq, n_eq], tp, tn, fp, fn, check=False, dict=True)

KeyboardInterrupt: 

In [148]:
sympy.GroebnerBasis([tmp5, score1_eq, n_eq, p_eq], tp, tn, fp, fn)

KeyboardInterrupt: 

In [112]:
score0_eq

dor*(n - tn)*(p - tp) - tn*tp

In [ ]:
gb.is_zero_dimensional

True

In [ ]:
gb

GroebnerBasis([-dor*mcc**2*n**2*p**2 - dor*n**3*p + fp**3*(dor**2*mcc**2*n + dor**2*p - 2*dor*mcc**2*n - 2*dor*p + mcc**2*n + p) + fp**2*(-dor**2*mcc**2*n**2 + dor**2*mcc**2*n*p - 2*dor**2*n*p + 3*dor*mcc**2*n**2 + 3*dor*n*p - 2*mcc**2*n**2 - mcc**2*n*p - n*p) + fn*(dor*mcc**2*n**2*p + dor*n**3) + fp*(-dor**2*mcc**2*n**2*p + dor**2*n**2*p - dor*mcc**2*n**3 + dor*mcc**2*n*p**2 + mcc**2*n**3 + mcc**2*n**2*p), -dor*mcc**2*n**2*p**2 - mcc**2*n**2*p**2 + dor*n**2*p**2 + n**2*p**2 + fn**2*(dor*mcc**2*n*p + dor*n**2 - mcc**2*n*p - n**2) + fp**2*(dor*mcc**2*n*p + dor*p**2 - mcc**2*n*p - p**2) + fn*(dor*mcc**2*n**2*p - dor*mcc**2*n*p**2 - 2*dor*n**2*p + mcc**2*n**2*p + mcc**2*n*p**2) + fp*(-dor*mcc**2*n**2*p + dor*mcc**2*n*p**2 - 2*dor*n*p**2 + mcc**2*n**2*p + mcc**2*n*p**2), fn*fp*(dor - 1) + fn*n + fp*p - n*p, fn - p + tp, fp - n + tn], tp, tn, fn, fp, domain='ZZ[p,n,dor,mcc]', order='grlex')

In [ ]:
gb[0]

-dor*mcc**2*n**2*p**2 - dor*n**3*p + fn*(dor*mcc**2*n**2*p + dor*n**3) + fp**3*(dor**2*mcc**2*n + dor**2*p - 2*dor*mcc**2*n - 2*dor*p + mcc**2*n + p) + fp**2*(-dor**2*mcc**2*n**2 + dor**2*mcc**2*n*p - 2*dor**2*n*p + 3*dor*mcc**2*n**2 + 3*dor*n*p - 2*mcc**2*n**2 - mcc**2*n*p - n*p) + fp*(-dor**2*mcc**2*n**2*p + dor**2*n**2*p - dor*mcc**2*n**3 + dor*mcc**2*n*p**2 + mcc**2*n**3 + mcc**2*n**2*p)

In [ ]:
gb.order

GradedLexOrder()

In [ ]:
print(sympy.Poly(gb[0], tp, tn, fp, fn).LM(order=gb.order))

tp**0*tn**0*fp**3*fn**0


In [ ]:
print(sympy.Poly(gb[0], tp, tn, fp, fn).LM())

tp**0*tn**0*fp**3*fn**0


In [ ]:
gb[0].coeff(sympy.Poly(gb[0], tp, tn, fp, fn).LM())

0

In [ ]:
sympy.Poly(2*fp**2*fn + fp*tp, tp, tn, fp, fn).as_expr().coeff(fn)

2*fp**2

In [ ]:
def collect_with_respect_to_vars(eq, vars):
    assert isinstance(vars, list)
    eq = eq.expand()
    if len(vars) == 0:
        return {1: eq}

    var_map = eq.collect(vars[0], evaluate=False)
    final_var_map = {}
    for var_power in var_map:
        sub_expression = var_map[var_power]
        sub_var_map = collect_with_respect_to_vars(sub_expression, vars[1:])
        for sub_var_power in sub_var_map:
            final_var_map[var_power*sub_var_power] = sub_var_map[sub_var_power]
    return final_var_map

In [ ]:
collect_with_respect_to_vars(2*fp**2*fn + fp*tp, [tp, tn, fp, fn])

{fp*tp: 1, fn*fp**2: 2}

In [ ]:
vars = [tp, fp, tn, fn]

In [ ]:
eq = 2*fp**2*fn + fp*tp

In [ ]:
eq.expand()

2*fn*fp**2 + fp*tp

In [ ]:
(2*fp**2*fn + fp*tp).collect(tp, evaluate=False)

{tp: fp, 1: 2*fn*fp**2}

In [ ]:
collect_with_respect_to_vars(gb[0], [tp, tn, fp, fn])

{fp**3: dor**2*mcc**2*n + dor**2*p - 2*dor*mcc**2*n - 2*dor*p + mcc**2*n + p,
 fp: -dor**2*mcc**2*n**2*p + dor**2*n**2*p - dor*mcc**2*n**3 + dor*mcc**2*n*p**2 + mcc**2*n**3 + mcc**2*n**2*p,
 fp**2: -dor**2*mcc**2*n**2 + dor**2*mcc**2*n*p - 2*dor**2*n*p + 3*dor*mcc**2*n**2 + 3*dor*n*p - 2*mcc**2*n**2 - mcc**2*n*p - n*p,
 fn: dor*mcc**2*n**2*p + dor*n**3,
 1: -dor*mcc**2*n**2*p**2 - dor*n**3*p}

In [ ]:
gb[0]

-dor*mcc**2*n**2*p**2 - dor*n**3*p + fn*(dor*mcc**2*n**2*p + dor*n**3) + fp**3*(dor**2*mcc**2*n + dor**2*p - 2*dor*mcc**2*n - 2*dor*p + mcc**2*n + p) + fp**2*(-dor**2*mcc**2*n**2 + dor**2*mcc**2*n*p - 2*dor**2*n*p + 3*dor*mcc**2*n**2 + 3*dor*n*p - 2*mcc**2*n**2 - mcc**2*n*p - n*p) + fp*(-dor**2*mcc**2*n**2*p + dor**2*n**2*p - dor*mcc**2*n**3 + dor*mcc**2*n*p**2 + mcc**2*n**3 + mcc**2*n**2*p)

In [ ]:
gb[0].args

(fn*(dor*mcc**2*n**2*p + dor*n**3),
 fp*(-dor**2*mcc**2*n**2*p + dor**2*n**2*p - dor*mcc**2*n**3 + dor*mcc**2*n*p**2 + mcc**2*n**3 + mcc**2*n**2*p),
 fp**2*(-dor**2*mcc**2*n**2 + dor**2*mcc**2*n*p - 2*dor**2*n*p + 3*dor*mcc**2*n**2 + 3*dor*n*p - 2*mcc**2*n**2 - mcc**2*n*p - n*p),
 fp**3*(dor**2*mcc**2*n + dor**2*p - 2*dor*mcc**2*n - 2*dor*p + mcc**2*n + p),
 -dor*n**3*p,
 -dor*mcc**2*n**2*p**2)

In [ ]:
gb[0].args[3]

dor**2*mcc**2*n + dor**2*p - 2*dor*mcc**2*n - 2*dor*p + mcc**2*n + p

In [ ]:
gb[1]

-dor*mcc**2*n**2*p**2 + dor*n**2*p**2 + fn**2*(dor*mcc**2*n*p + dor*n**2 - mcc**2*n*p - n**2) + fn*(dor*mcc**2*n**2*p - dor*mcc**2*n*p**2 - 2*dor*n**2*p + mcc**2*n**2*p + mcc**2*n*p**2) + fp**2*(dor*mcc**2*n*p + dor*p**2 - mcc**2*n*p - p**2) + fp*(-dor*mcc**2*n**2*p + dor*mcc**2*n*p**2 - 2*dor*n*p**2 + mcc**2*n**2*p + mcc**2*n*p**2) - mcc**2*n**2*p**2 + n**2*p**2

In [ ]:
print(sympy.Poly(gb[1], tp, tn, fp, fn).LM(order=gb.order))

tp**0*tn**0*fp**2*fn**0


In [ ]:
gb[1].coeff(fp**2)

dor*mcc**2*n*p + dor*p**2 - mcc**2*n*p - p**2

In [ ]:
gb[2]

fn*fp*(dor - 1) + fn*n + fp*p - n*p

In [ ]:
print(sympy.Poly(gb[4], tp, tn, fp, fn).LM(order=gb.order))

tp**0*tn**1*fp**0*fn**0


In [ ]:
gb[2].coeff(fp*fn)

dor - 1

In [ ]:
gb[3]

fn - p + tp

In [ ]:
print(sympy.Poly(gb[3], tp, tn, fp, fn).LM(order=gb.order))

tp**1*tn**0*fp**0*fn**0


In [ ]:
gb[3].coeff(tp)

1

In [ ]:
print(sympy.Poly(gb[4], tp, tn, fp, fn).LM())

tp**0*tn**1*fp**0*fn**0


In [ ]:
gb[4].coeff(tn)

1

In [ ]:
gb[1]

-dor*mcc**2*n**2*p**2 + dor*n**2*p**2 + fn**2*(dor*mcc**2*n*p + dor*n**2 - mcc**2*n*p - n**2) + fn*(dor*mcc**2*n**2*p - dor*mcc**2*n*p**2 - 2*dor*n**2*p + mcc**2*n**2*p + mcc**2*n*p**2) + fp**2*(dor*mcc**2*n*p + dor*p**2 - mcc**2*n*p - p**2) + fp*(-dor*mcc**2*n**2*p + dor*mcc**2*n*p**2 - 2*dor*n*p**2 + mcc**2*n**2*p + mcc**2*n*p**2) - mcc**2*n**2*p**2 + n**2*p**2

In [ ]:
gb[2]

fn*fp*(dor - 1) + fn*n + fp*p - n*p

In [ ]:
gb[3]

fn - p + tp

In [ ]:
gb[4]

fp - n + tn

In [ ]:
sympy.solve([score0_eq, score1_eq, n_eq, p_eq], tp, tn, fn, fp, dict=True)

[{fn: 0, fp: n, tn: 0, tp: p},
 {fn: p, fp: 0, tn: n, tp: 0},
 {fn: (-dor*mcc**2*n*p + dor*n*p - mcc**2*n*p - 2*mcc**2*p**2 - n*p - sqrt(n*p*(dor**2*mcc**4*n*p - 2*dor**2*mcc**2*n*p + dor**2*n*p - 2*dor*mcc**4*n*p - 4*dor*mcc**2*n**2 - 4*dor*mcc**2*n*p - 4*dor*mcc**2*p**2 - 2*dor*n*p + mcc**4*n*p - 2*mcc**2*n*p + n*p)))/(2*(dor*mcc**2*p + dor*n - mcc**2*p - n)),
  fp: (-dor*mcc**2*n*p + dor*n*p - 2*mcc**2*n**2 - mcc**2*n*p - n*p + sqrt(dor**2*mcc**4*n**2*p**2 - 2*dor**2*mcc**2*n**2*p**2 + dor**2*n**2*p**2 - 2*dor*mcc**4*n**2*p**2 - 4*dor*mcc**2*n**3*p - 4*dor*mcc**2*n**2*p**2 - 4*dor*mcc**2*n*p**3 - 2*dor*n**2*p**2 + mcc**4*n**2*p**2 - 2*mcc**2*n**2*p**2 + n**2*p**2))/(2*dor*mcc**2*n + 2*dor*p - 2*mcc**2*n - 2*p),
  tn: (2*dor*mcc**2*n**2 + dor*mcc**2*n*p + dor*n*p + mcc**2*n*p - n*p - sqrt(n*p*(dor**2*mcc**4*n*p - 2*dor**2*mcc**2*n*p + dor**2*n*p - 2*dor*mcc**4*n*p - 4*dor*mcc**2*n**2 - 4*dor*mcc**2*n*p - 4*dor*mcc**2*p**2 - 2*dor*n*p + mcc**4*n*p - 2*mcc**2*n*p + n*p)))/(2*(dor*mcc**

In [ ]:
print(sympy.python(gb[0].coeff(fp**3)))

dor = Symbol('dor')
mcc = Symbol('mcc')
n = Symbol('n')
p = Symbol('p')
e = dor**2*mcc**2*n + dor**2*p - 2*dor*mcc**2*n - 2*dor*p + mcc**2*n + p


In [ ]:
dor = score0_sym
mcc = score1_sym
p = symbols['p']
n = symbols['n']

In [ ]:
sympy.solve(dor**2*mcc**2*n + dor**2*p - 2*dor*mcc**2*n - 2*dor*p + mcc**2*n + p)

[{dor: 1}, {n: -p/mcc**2}]

In [ ]:
#ji*ppv - ji + ppv = 0
#ji*ppv = ji - ppv

In [ ]:
ji*ppv - ji + ppv = 

In [ ]:
sympy.solve(score1_exp * score0_exp - score1_exp + score0_exp)

[{p: 0}, {tp: 0}]

In [ ]:
sympy.GroebnerBasis([npv_eq, n_eq, p_eq], tp, tn, fn, fp, order='lex')

GroebnerBasis([fp*(npv - 1) - n*npv + n - npv*p + npv*tp, fp - n + tn, fn*npv + fp*(1 - npv) + n*npv - n], tp, tn, fn, fp, domain='ZZ[p,n,npv]', order='lex')

In [ ]:
sympy.GroebnerBasis([spec_eq, npv_eq, n_eq, p_eq], tp, tn, fn, fp, order='lex').is_zero_dimensional

True

In [ ]:
subst = {npv: 0}
spec_eq_new = spec_eq.subs(subst)
npv_eq_new = npv_eq.subs(subst)
n_eq_new = n_eq.subs(subst)
p_eq_new = p_eq.subs(subst)

In [ ]:
sympy.GroebnerBasis([spec_eq_new, npv_eq_new, n_eq_new, p_eq_new], tp, tn, fn, fp, order='lex')

GroebnerBasis([1], tp, tn, fn, fp, domain='ZZ[p,n,spec]', order='lex')

In [ ]:
sympy.GroebnerBasis([spec_eq_new, npv_eq_new, n_eq_new, p_eq_new], tp, tn, fn, fp, order='lex').is_zero_dimensional

False

In [ ]:
spec_eq_new, npv_eq_new, n_eq_new, p_eq_new

((n + p)*(n*spec - tn)/n, -tn, -fp + n - tn, -fn + p - tp)

In [ ]:
sympy.GroebnerBasis([acc_eq_new, npv_eq_new, n_eq_new, p_eq_new], tp, tn, fn, fp, order='lex')

GroebnerBasis([-acc*n - acc*p + tp, tn, acc*n + acc*p + fn - p, fp - n], tp, tn, fn, fp, domain='ZZ[p,n,acc]', order='lex')

In [ ]:
sympy.solve_poly_system([acc_eq, npv_eq, n_eq, p_eq], tp, tn, fp, fn)

[((acc*n*npv - acc*n + acc*npv*p - acc*p + npv*p)/(2*npv - 1),
  npv*(acc*n + acc*p - p)/(2*npv - 1),
  -(acc*n*npv + acc*npv*p - 2*n*npv + n - npv*p)/(2*npv - 1),
  -(npv - 1)*(acc*n + acc*p - p)/(2*npv - 1))]